In [1]:
from aut import *
from pyspark.sql.functions import col, explode_outer

In [2]:
webpages = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities") \
  .webpages() \
  .select(remove_prefix_www(extract_domain("url")).alias("domain"), "url", "crawl_date", explode_outer(extract_links("url", "content")).alias("link")) \
  .filter(col("content").like("%food%")) \
  .select("url", "domain", "crawl_date", col("link._1").alias("destination_page"))

In [3]:
webpages.show(5, False)

+-------------------------------------------------------------------------------------+-------------+----------+--------------------------------------------+
|url                                                                                  |domain       |crawl_date|destination_page                            |
+-------------------------------------------------------------------------------------+-------------+----------+--------------------------------------------+
|http://geocities.com/kelsoonbutler/boardDocs/BoardMeetingMinutes_AGM_March12_2002.htm|geocities.com|20091027  |null                                        |
|http://geocities.com/krooyaimaha/health5.htm                                         |geocities.com|20091027  |http://geocities.com/krooyaimaha/health5.htm|
|http://geocities.com/krooyaimaha/health5.htm                                         |geocities.com|20091027  |http://geocities.com/krooyaimaha/health5.htm|
|http://geocities.com/krooyaimaha/health5.htm       